In [ ]:
!pip install -U featuretools
!pip install nvcc4jupyter
#!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc4jupyter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.2/619.2 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 754.4/754.4 kB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.1/236.1 kB 28.7 MB/s eta 0:00:00
  Attempting uninstall: holidays
    Found existing installation: holidays 0.41
    Uninstalling holidays-0.41:
      Successfully uninstalled holidays-0.41
Source files will be saved in "/tmp/tmpbyz7ujpy".


In [58]:
%%writefile main.cu
#include <stdio.h>
#include <iostream>
#include <stdexcept>
#include <cassert>
#include <stdexcept>
#include <iomanip>
#include <vector>

float round_to_one_decimal_place(float number) {
  return (static_cast<float>(static_cast<int>(number * 10.)) / 10.);
}

float round_to_three_decimal_place(float number) {
  return (static_cast<float>(static_cast<int>(number * 1000.)) / 1000.);
}

__device__ float* get_row(float* values, int row, int n) {

    float* row_values = (float*) malloc(sizeof(float) * n);
    for (int i = 0; i < n; i++) {
      int values_index = (row * n) + i;
      row_values[i] = values[values_index];
    }
    return row_values;
  }

  __device__ float* get_column(float* values, int column, int n) {

    float* column_values = (float*) malloc(sizeof(float) * n);
    for (int i = 0; i < n; i++) {
      int values_index = column + (i * n);
      column_values[i] = values[values_index];
    }
    return column_values;
  }

  __device__ float dot_product(float* v1, float* v2, int n) {

    float result = 0.0;
    for (int i = 0; i < n; i++) {
      result += (v1[i] * v2[i]);
    }

    return result;
  }

  __device__ void multiply_by_constant(float* values, int size, double constant) {

    for (int i = 0; i < size; i++) {
      values[i] = values[i] * constant;
    }

  }

  __device__ float* bracket_w(float x, float y, float z) {
    /* See page 113 of Modern Robotics.
    */

    int num_entries_in_matrix = 9;
    float* values = (float*) malloc(sizeof(float) * num_entries_in_matrix);

    // First row.
    values[0] = 0.;
    values[1] = -z;
    values[2] = y;

    // Second row.
    values[3] = z;
    values[4] = 0;
    values[5] = -x;

    // Third row.
    values[6] = -y;
    values[7] = x;
    values[8] = 0.;

    return values;
  }

  __device__ float* bracket_w_squared(float x, float y, float z) {
    /* See page 113 of Modern Robotics.
    * https://www.symbolab.com/solver/matrix-multiply-calculator
    */
    float* values = (float*) malloc(sizeof(float) * 9);

    // First row.
    values[0] = -(z * z) - (y * y);
    values[1] = y * x;
    values[2] = z * x;

    // Second row.
    values[3] = y * x;
    values[4] = -(z * z) - (x * x);
    values[5] = z * y;

    // Third row.
    values[6] = z * x;
    values[7] = z * y;
    values[8] = -(y * y) - (x * x);

    return values;
  }

  struct Twist {
  Twist(): wx(0.), wy(0.), wz(0.), vx(0.), vy(0.), vz(0.) {

  }

  Twist(float wx, float wy, float wz, float vx, float vy, float vz): wx(wx), wy(wy), wz(wz), vx(vx), vy(vy), vz(vz) {

  }

  private:
    float wx;
    float wy;
    float wz;
    float vx;
    float vy;
    float vz;
};

struct Screw {
  Screw(): wx(0.), wy(0.), wz(0.), vx(0.), vy(0.), vz(0.) {

  }

  Screw(float wx, float wy, float wz, float vx, float vy, float vz): wx(wx), wy(wy), wz(wz), vx(vx), vy(vy), vz(vz) {

  }

  __device__ Twist create_twist(float theta) {
    return Twist(wx * theta, wy * theta, wz * theta, vx * theta, vy * theta, vz * theta);
  }

  public:
    float wx;
    float wy;
    float wz;
    float vx;
    float vy;
    float vz;
};

  __device__ float* exponential_rotation(float x, float y, float z, float theta) {

    // I + sin θ[ωˆ] + (1 − cos θ)[ωˆ]2
    // A + B + C

    int num_entries_in_matrix = 9;
    float* B = bracket_w(x, y, z);
    float* C = bracket_w_squared(x, y, z);
    multiply_by_constant(B, num_entries_in_matrix, sin(theta));
    multiply_by_constant(C, num_entries_in_matrix, 1.0 - cos(theta));

    // Add in the component from the identity matrix.
    B[0] += 1.0;
    B[4] += 1.0;
    B[8] += 1.0;

    for (int i = 0; i < num_entries_in_matrix; i++) {
      B[i] += C[i];
    }

    // Cleanup memory.
    free(C);

    return B;
  }

   __device__ float* rigid_body_exponential_translation(float wx, float wy, float wz, float vx, float vy, float vz, float theta) {

      // (Iθ + (1 − cos θ)[ω] + (θ − sin θ)[ω]2)v
      // (A + B + C) * (v)
      // (3 x 3 matrix) * (3 x 1 matrix) = (3 x 1 matrix)

      float* result = (float*) malloc(sizeof(float) * 3);
      int num_entries_in_matrix = 9;
      float* B = bracket_w(wx, wy, wz);
      float* C = bracket_w_squared(wx, wy, wz);
      multiply_by_constant(B, num_entries_in_matrix, 1.0 - std::cos(theta));
      multiply_by_constant(C, num_entries_in_matrix, theta - std::sin(theta));

      // Add the component due to A.
      B[0] += theta;
      B[4] += theta;
      B[8] += theta;

      // Add C.
      for (int i = 0; i < num_entries_in_matrix; i++) {
        B[i] += C[i];
      }

      // Multiply.
      result[0] = (B[0] * vx) + (B[1] * vy) + (B[2] * vz);
      result[1] = (B[3] * vx) + (B[4] * vy) + (B[5] * vz);
      result[2] = (B[6] * vx) + (B[7] * vy) + (B[8] * vz);

      return result;
   }

  __device__ void normalize(float& x, float& y, float& z) {

    float magnitude = (x * x) + (y * y) + (z * z);
    x = x / magnitude;
    y = y / magnitude;
    z = z / magnitude;
  }

__device__ float* rigid_body_exponential(float wx, float wy, float wz, float vx, float vy, float vz, float theta) {

    // See page 113 of Modern Robotics.

    int num_entries_in_matrix = 4 * 4;
    float* result = (float*) malloc(sizeof(float) * num_entries_in_matrix);

    normalize(wx, wy, wz);
    float* rotation_matrix = exponential_rotation(wx, wy, wz, theta);
    float* translation = rigid_body_exponential_translation(wx, wy, wz, vx, vy, vz, theta);

    // Write the rotation matrix to the result.
    result[0] = rotation_matrix[0];
    result[1] = rotation_matrix[1];
    result[2] = rotation_matrix[2];

    result[4] = rotation_matrix[3];
    result[5] = rotation_matrix[4];
    result[6] = rotation_matrix[5];

    result[8] = rotation_matrix[6];
    result[9] = rotation_matrix[7];
    result[10] = rotation_matrix[8];

    // Write the translation component to the result.
    result[3] = translation[0];
    result[7] = translation[1];
    result[11] = translation[2];

    // Write the last row of the matrix.
    result[12] = 0.0;
    result[13] = 0.0;
    result[14] = 0.0;
    result[15] = 1.0;

    // Free memory.

    return result;
  }

__device__ float* rigid_body_exponential(Screw* screw, float theta) {
    return rigid_body_exponential(screw->wx, screw->wy, screw->wz, screw->vx, screw->vy, screw->vz, theta);
}

struct Vector3 {
  Vector3(): x(0.0), y(0.0), z(0.0) {

  }

  Vector3(float x, float y, float z): x(x), y(y), z(z) {

  }

  private:
    float x;
    float y;
    float z;
};


__device__ void multiply_on_device(float* left, float* right, int n, float* result) {

  for (int row_index = 0; row_index < n; row_index++) {
    for (int col_index = 0; col_index < n; col_index++) {

      int result_index = (n * row_index) + col_index;
      float* v1 = get_row(left, row_index, n);
      float* v2 = get_column(right, col_index, n);
      result[result_index] = dot_product(v1, v2, n);
    }
  }
}


__global__ void multiply(float* left, float* right, int n, float* result) {

  multiply_on_device(left, right, n, result);
}

__global__ void rigid_body_exponential_rotation_kernel(Screw* screw_axes, float* thetas, int num_joints, float** outputs) {


}

struct SquareMatrix {

  int n;
  float* host_values;
  float* device_values;
  ~SquareMatrix() {
    //free(host_values);
    cudaFree(device_values);
  }

  SquareMatrix(int n): n(n) {

    // Allocate space on the host.
    host_values = (float*) malloc(sizeof(float) * n * n);

    // Allocate space on the device.
    cudaMalloc((void**)&device_values, sizeof(float) * n * n);

    for (int i = 0; i < n; i++) {
      for (int j = 0; j < n; j++) {
        int index = (i * n) + j;
        if (i == j) {
          host_values[index] = 1.0;
        }
        else {
          host_values[index] = 0.0;
        }
      }
    }
  }



  SquareMatrix write(float value, int row, int column) {

    int linear_index = (row * n) + column;
    if (linear_index < 0 || linear_index > (n * n)) {
      throw std::invalid_argument("Illegal (row, column) pair.");
    }

    host_values[linear_index] = value;
  }

  __device__ SquareMatrix gpu_write(float value, int row, int column) {

    int linear_index = (row * n) + column;
    host_values[linear_index] = value;
  }

  void print() {

    for (int i = 0; i < n; i++) {
      for (int j = 0; j < n; j++) {
        int linear_index = (i * n) + j;
        float rounded_value = round_to_three_decimal_place(host_values[linear_index]);
        std::cout << rounded_value << " ";
      }
      std::cout << std::endl;
    }
  }

  __device__ void gpu_write_row(float* values, int row) {

    int start_linear_index = (row * n);
    for (int i = 0; i < n; i++) {
      int device_index = start_linear_index + i;
      device_values[device_index] = values[i];
    }
  }

  void write_row(float* values, int row) {

    int start_linear_index = (row * n);
    for (int i = 0; i < n; i++) {
      int host_index = start_linear_index + i;
      host_values[host_index] = values[i];
    }
  }

  float get_value(int row, int column) const {

      int linear_index = (row * n) + column;
      if (linear_index > (n * n) || linear_index < 0) {
        throw std::invalid_argument("Illegal (row, column) pair.");
      }
      return host_values[linear_index];
  }

  float* get_row(int row) const {

    float* row_values = (float*) malloc(sizeof(float) * n);
    for (int i = 0; i < n; i++) {
      row_values[i] = get_value(row, i);
    }
    return row_values;
  }

  float* get_column(int column) const {
    float* column_values = (float*) malloc(sizeof(float) * n);
    for (int i = 0; i < n; i++) {
      column_values[i] = get_value(i, column);
    }
    return column_values;
  }

  float dot_product(const SquareMatrix& right, int row, int column) const {

    float* row_values = get_row(row);
    float* column_values = right.get_column(column);

    float result = 0.0;
    for (int i = 0; i < n; i++) {
      result += (row_values[i] * column_values[i]);
    }

    free(row_values);
    free(column_values);
    return result;
  }

  SquareMatrix operator*(const SquareMatrix& right) {

    // Multiply the matrices on the CPU.
    SquareMatrix result = SquareMatrix(right.n);

    for (int row = 0; row < n; row++) {
      for (int column = 0; column < n; column++) {

        float value = dot_product(right, row, column);
        result.write(value, row, column);
      }
    }

    return result;
  }

  SquareMatrix gpu_multiply(const SquareMatrix& right) {
    // Multiply the matrices on the GPU.
    SquareMatrix result = SquareMatrix(right.n);

    // Copy the values onto the GPU.
    cudaMemcpy(device_values, host_values, sizeof(float) * n * n, cudaMemcpyHostToDevice);
    cudaMemcpy(right.device_values, right.host_values, sizeof(float) * n * n, cudaMemcpyHostToDevice);
    cudaMemcpy(result.device_values, result.host_values, sizeof(float) * n * n, cudaMemcpyHostToDevice);

    // Run the kernel.
    dim3 blocks(2, 2);
    dim3 threads(80, 8);
    multiply<<<blocks, threads>>>(device_values, right.device_values, n, result.device_values);

    // Copy the result back from the device.
    cudaMemcpy(result.host_values, result.device_values, sizeof(float) * n * n, cudaMemcpyDeviceToHost);

    return result;
  }
};

struct Transformation: public SquareMatrix {

  Transformation(): SquareMatrix(4) {
  }

  Transformation(float* values): SquareMatrix(4) {

    for (int i = 0; i < n * n; i++) {
      host_values[i] = values[i];
    }
  }

  Transformation operator*(const Transformation& right) {

    // Multiply the matrices on the CPU.
    Transformation result = Transformation();

    for (int row = 0; row < n; row++) {
      for (int column = 0; column < n; column++) {

        float value = dot_product(right, row, column);
        result.write(value, row, column);
      }
    }

    return result;
  }

  Transformation gpu_multiply(const Transformation& right) {
    // Multiply the matrices on the GPU.
    Transformation result = Transformation();

    // Copy the values onto the GPU.
    cudaMemcpy(device_values, host_values, sizeof(float) * n * n, cudaMemcpyHostToDevice);
    cudaMemcpy(right.device_values, right.host_values, sizeof(float) * n * n, cudaMemcpyHostToDevice);
    cudaMemcpy(result.device_values, result.host_values, sizeof(float) * n * n, cudaMemcpyHostToDevice);

    // Run the kernel.
    dim3 blocks(2, 2);
    dim3 threads(80, 8);
    multiply<<<blocks, threads>>>(device_values, right.device_values, n, result.device_values);

    // Copy the result back from the device.
    cudaMemcpy(result.host_values, result.device_values, sizeof(float) * n * n, cudaMemcpyDeviceToHost);

    return result;
  }
};

void create_joint_angles(float a, float b, float c, float d, float e, float f, float* output) {
  output[0] = a;
  output[1] = b;
  output[2] = c;
  output[3] = d;
  output[4] = e;
  output[5] = f;
}


  __device__ float* forward_kinematics(Screw* screw_axes, float* thetas, float* M, int num_joints) {

    // Allocate space for the transformations.
    int num_entries_in_transformation_matrix = 4 * 4;
    float** transformations = (float**) malloc(sizeof(float*) * (num_joints + 1));

    // Create the transformations.
    for (int i = 0; i < num_joints; i++) {
      transformations[i] = rigid_body_exponential(screw_axes + i, thetas[i]);
    }

    // Add M to the list of transformations.
    transformations[num_joints] = M;

    float* left = transformations[0];
    float* result = (float*) malloc(sizeof(float) * num_entries_in_transformation_matrix);
    for (int i = 0; i < num_joints + 1; i++) {

       multiply_on_device(left, transformations[i], 4, result);

       // Copy the result back to the left.
       for (int j = 0; j < num_entries_in_transformation_matrix; j++) {
          left[j] = result[j];
       }
    }

    return left;
  }

  __global__ void forward_kinematics_kernel(Screw* screw_axes, float* thetas, float* M, int num_joints, float* outputs) {

    int theta_index = threadIdx.x;
    int outputs_index = threadIdx.x;
    int num_entries_in_transformation_matrix = 16;
    float* result = forward_kinematics(screw_axes, thetas + (num_joints * theta_index), M, num_joints);

    // Copy the result into the outputs.
    int constant_offset = num_entries_in_transformation_matrix * outputs_index;

    for (int i = 0; i < num_entries_in_transformation_matrix; i++) {
      outputs[constant_offset + i] = result[i];
    }

  }


#define N 1000000
#define num_threads_per_block 1024

int main() {

  // I know the screw axis and the angle of rotation about it.
  // How do I recover the transformation matrix?
  // Then I multipy all the transformation matrices in order.
  // Matrix exponential.
  // See page 105 of Modern Robotics
  // https://hades.mech.northwestern.edu/images/7/7f/MR.pdf
  // Page 113 contains the exponetial equation of [w]* theta

  // Robot info.
  int num_joints = 6;
  int num_configurations = 200;

  // Create the output arrays.
  float num_outputs = 16 * num_configurations;
  float* outputs_host = (float*) malloc(sizeof(float) * num_outputs);
  float* outputs_device;
  cudaMalloc((void**)&outputs_device, sizeof(float) * num_outputs);
  for (int i = 0; i < num_outputs; i++) {
    outputs_host[i] = 0.;
  }

  // Copy the outputs onto the device.
  cudaMemcpy(outputs_device, outputs_host, sizeof(float) * 16 * num_configurations, cudaMemcpyHostToDevice);

  Screw* screw_axes_host = (Screw*) malloc(sizeof(Screw) * num_joints);
  Screw* screw_axes_device;
  cudaMalloc((void**)&screw_axes_device, sizeof(Screw) * num_joints);

  int num_entries_in_transformation_matrix = 4 * 4;
  float* M_host = (float*) malloc(sizeof(float) * num_entries_in_transformation_matrix);
  float* M_device;
  cudaMalloc((void**)&M_device, sizeof(float) * num_entries_in_transformation_matrix);

  float* thetas_host = (float*) malloc(sizeof(float) * num_joints * num_configurations);
  float* thetas_device;
  cudaMalloc((void**)&thetas_device, sizeof(float) * num_joints * num_configurations);

  // UR5 Screw axes.
  // See page 148 of Modern Robotics.
  float H1 = 0.089;
  float H2 = 0.095;
  float L1 = 0.425;
  float L2 = 0.392;
  float W1 = 0.109;
  float W2 = 0.082;

  // Define the screw axes.
  screw_axes_host[0] = Screw(0., 0., 1., 0., 0., 0.);
  screw_axes_host[1] = Screw(0., 1., 0., -H1, 0., 0.);
  screw_axes_host[2] = Screw(0., 1., 0., -H1, 0., L1);
  screw_axes_host[3] = Screw(0., 1., 0., -H1, 0., L1 + L2);
  screw_axes_host[4] = Screw(0., 0., -1., -W1, L1 + L2, 0.);
  screw_axes_host[5] = Screw(0., 1., 0., H2 - H1, 0., L1 + L2);

  // Define the home pose.
  M_host[0] = -1.;
  M_host[1] = 0.;
  M_host[2] = 0.;
  M_host[3] = L1 + L2;

  M_host[4] = 0.;
  M_host[5] = 0.;
  M_host[6] = 1.;
  M_host[7] = W1 + W2;

  M_host[8] = 0.;
  M_host[9] = 1.;
  M_host[10] = 0.;
  M_host[11] = H1 - H2;

  M_host[12] = 0.;
  M_host[13] = 0.;
  M_host[14] = 0.;
  M_host[15] = 1.;

  for (int i = 0; i < num_configurations; i++) {
    // create_joint_angles(3.14/2.0, 3.14/2.0, 0., 0., 0., 0., thetas_host + (num_joints * i));

    create_joint_angles(0., -3.14/2.0, 0., 0., 3.14/2.0, 0., thetas_host + (num_joints * i));
  }

  // Copy the screw axes onto the device.
  cudaMemcpy(screw_axes_device, screw_axes_host, sizeof(Screw) * num_joints, cudaMemcpyHostToDevice);

  // Copy M onto the device.
  cudaMemcpy(M_device, M_host, sizeof(float) * num_entries_in_transformation_matrix, cudaMemcpyHostToDevice);

  // Copy the joint angles onto the device.
  cudaMemcpy(thetas_device, thetas_host, sizeof(float) * num_joints * num_configurations, cudaMemcpyHostToDevice);

  // forward_kinematics_kernel(Screw* screw_axes, float** thetas, int num_joints, float* outputs)
  forward_kinematics_kernel<<<1, num_configurations>>>(screw_axes_device, thetas_device, M_device, num_joints, outputs_device);

  // Copy the outputs back to the host.
  int a = cudaMemcpy(outputs_host, outputs_device, sizeof(float) * 16 * num_configurations, cudaMemcpyDeviceToHost);

  std::cout << a << std::endl;

  // Convert the outputs back into Transformations.
  std::vector<Transformation> transformations = std::vector<Transformation>();
  for (int i = 0; i < num_configurations; i++) {
    float* matrix = outputs_host + (16 * i);
    Transformation result = Transformation(matrix);
    transformations.push_back(result);
  }

  // Display one of the Transformations.
  transformations[num_configurations - 1].print();

  cudaFree(screw_axes_device);
  cudaFree(thetas_device);
  cudaFree(outputs_device);
  cudaFree(M_device);

  // Free the host memory.
  free(screw_axes_host);
  free(outputs_host);
  free(M_host);

  return 0;
}

Overwriting main.cu


In [59]:
%%script bash
nvcc main.cu -o forward_kinematics

In [60]:
%%script bash
./forward_kinematics

0
0 -0.999 0 0.095 
0.999 0 0 0.109 
0 0 0.999 0.987 
0 0 0 1 
